In [1]:
import pandas as pd
from scipy import sparse
import numpy as np
import random
from tqdm import tqdm_notebook

In [53]:
print("WOW")

WOW


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.stem.snowball import SnowballStemmer
import langid

In [ ]:
df = pd.read_csv("lyrics.csv").set_index("index")

In [ ]:
df = df.dropna()
df = df.query("genre != 'Not Available'")
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head(10)

In [ ]:
lyricsList = []

nilai = df['lyrics'].values
language = []
for row in tqdm_notebook(nilai):
    word = row.replace("\n"," ")
    lyricsList.append(word)
    language.append(langid.classify(word)[0])
df['lyrics'] = lyricsList

In [ ]:
df['language'] = language

In [ ]:
filtered = df.query("language == 'en'")

In [ ]:
filtered

In [ ]:
df.loc[50]['lyrics']

In [ ]:
set(df['genre'].values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
vectorizer = CountVectorizer(min_df=25)

In [ ]:
result = vectorizer.fit_transform(df['lyrics'].values)

In [ ]:
result.shape

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [ ]:
label = encoder.fit_transform(df[['genre']].values).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(result, df[['genre']].values):
    train_X = result[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = result[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=25)

In [ ]:
tfidf_result = tfidf_vectorizer.fit_transform(df['lyrics'].values)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(tfidf_result, df[['genre']].values):
    train_X = tfidf_result[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = tfidf_result[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
sum([0.5495909493683928, 0.5490066771082351, 0.5477402468932259, 0.5482255471744776, 0.546919763391661]) / 5

In [ ]:
tfidf_result.shape

In [ ]:
title = np.array([" ".join(I.split("-")) for I in df["song"].values])

In [ ]:
title_vectorizer = TfidfVectorizer(min_df=25)
title_result = tfidf_vectorizer.fit_transform(title)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(result, df[['genre']].values):
    train_X = title_result[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = title_result[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
sum([0.4687081418591712, 0.4697469293545462, 0.46728355625167445, 0.46766415234326697, 0.46895030812671323]) / 5

In [ ]:
W = np.random.normal(size=(result.shape[1], 11))
b = np.random.normal(size=(11,))

In [ ]:
def softmax(inputs):
    exp_result =np.exp(inputs)
    return exp_result / float(max(sum(exp_result), 0.00001))

def model(X, W, b):
    y_linear = np.dot(X, W) + b
    return softmax(y_linear)

def calculate_l2(X, b, delta):
    l2 = b
    l2 += X.sum()
    return l2 * delta

In [ ]:
def sgd_logistic_regression(X, y, W_aksen, b_aksen, lr, delta):
    prediction = model(X, W_aksen, b_aksen)
    shape = W_aksen[:,0].shape[0]
    error = prediction - y
    for kelas in range(len(prediction)):
        l2 = calculate_l2(W_aksen[:,kelas], b_aksen[kelas], delta)
        b_aksen[kelas] -= lr * (error[kelas] + l2)
        W_aksen[:, kelas] -= lr * (X * error[kelas] + l2)

In [ ]:
for row in tqdm_notebook(range(30000)):
    sgd_logistic_regression(np.array(tfidf_result[row].todense())[0], label[row], W, b, 0.01, 1 / 20000)

In [ ]:
insertedWord = []
for I in range(W.shape[0]):
    if np.sum(np.abs(W[I,:]) > 1) > 1:
        insertedWord.append(I)

In [ ]:
len(insertedWord)

In [ ]:
tfidf_result_filtered = tfidf_result[:, insertedWord]

In [ ]:
tfidf_result_filtered

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(tfidf_result_filtered, df[['genre']].values):
    train_X = tfidf_result_filtered[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = tfidf_result_filtered[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
sum([0.5467059575081913, 0.5483266012694749, 0.5464418935350246, 0.5459997526895016, 0.5457861868546343]) / 5

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
sentimentScore = []
for I in tqdm_notebook(lyricsList):
    ss = sid.polarity_scores(I)
    sentimentScore.append(ss)

In [ ]:
sentimentLabel = []

for I in tqdm_notebook(sentimentScore):
    if I['compound'] > 0:
        sentimentLabel.append([1,0])
    else:
        sentimentLabel.append([0,1])

In [ ]:
sentimentLabel = np.array(sentimentLabel)

In [ ]:
combined = sparse.hstack((tfidf_result_filtered,sentimentLabel))

In [ ]:
combined = combined.tocsr()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(combined, df[['genre']].values):
    train_X = combined[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = combined[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
sum([0.5460877449667195, 0.5506965625257605, 0.5496156461884055, 0.5502040311611228, 0.549475463220594]) / 5

In [ ]:
combined = sparse.hstack((tfidf_result,sentimentLabel))
combined = combined.tocsr()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(combined, df[['genre']].values):
    train_X = combined[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = combined[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))

In [ ]:
combined = sparse.hstack((title_result,sentimentLabel))
combined = combined.tocsr()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for train, test in kf.split(combined, df[['genre']].values):
    train_X = combined[train]
    train_y = df['genre'].values[train]
    
    model = MultinomialNB()
    model.fit(train_X, train_y)
    
    test_X = combined[test]
    test_y = df['genre'].values[test]
    
    prediction = model.predict(test_X)
    accuracy = 0
    for I in range(len(prediction)):
        if prediction[I] == test_y[I]:
            accuracy += 1
    print(accuracy / len(test_y))